# Web Scraping con Selenium

##### Vamos a ver como acceder a una web creando un bot con la librería Selenium que interactuará con la página


## 1. Instalación de librerías

Tenemos que usar las librerías Selenium y Webdriver_manager. Vamos a instalarlas:

In [263]:
#%pip install selenium
#%pip install webdriver_manager
#%pip install --upgrade pandas


## 2. Importación de las librerías

In [264]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time


## 3. Instalación del webdriver

El webdriver es lo que nos va a permitir conectarnos con el navegador. Lo instalamos:

In [265]:
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

## 4. Abrir web
Abrimos la web en el driver ya podemos navegar y acceder a los elementos de la página

In [266]:
url = "https://www.filmaffinity.com/"
driver.get(url)

## 5. Scrapeo
Empezamos lo interesante:

Podemos acceder a elementos en la página de varias maneras:
- Nombre de la etiqueta
- Atributo: Clase
- Atributo: ID
- Atributo: Name
- Selector: Xpath
- Selector: CSS Selector

Para ello vamos a usar el driver que hemos creado y el By.

##### Beginner Selenium Cheatsheet:
Sacar un elemento:
- element = driver.find_element(by, value)

Sacar varios elementos:
- element = driver.find_elements(by, value)

Sacar atributos:
- attribute = element.--el atributo--
- attribute = element.get_attribute(--el atributo--)

Hacer click:
- element.click()

Teclear:
- element.send_keys()

Gestión de pestañas:
- driver.switch_to.window(driver.window_handles[-1])
- driver.get(url)
- driver.close()

Vemos que hay un popup pidiendo que aceptemos las cookies. ¡Vamos a aceptarlo!

In [267]:
elements_by_tag = driver.find_elements(By.TAG_NAME, 'button')
print(type(elements_by_tag))
for i in elements_by_tag:
    print(i.text)
#elements_by_tag[2].click()

<class 'list'>
socios
MÁS OPCIONES
NO ACEPTO
ACEPTO


### Dos formas de obtener todos los botones

Recorriendo la lista

In [268]:
# Encontrar elementos por clase
#element_by_class_name = driver.find_element(By.CLASS_NAME,'css-xlut8b')
#print(element_by_class_name.find_element(By.TAG_NAME, 'span').text)
# Encontrar elementos por XPATH
#element_by_xpath = driver.find_element(By.XPATH,'//*[@id="qc-cmp2-ui"]/div[2]/div/button[3]')
#print(element_by_xpath.text)

Recorriendo los índices

In [269]:
accept = driver.find_element(By.CLASS_NAME, "css-xlut8b")
#Nos aseguramos de que es el botón que estamos buscando
print("Etiqueta: {}".format(accept.tag_name))
print("Texto de la etiqueta: {}".format(accept.text))
print("Atributo mode: {}".format(accept.get_attribute('mode')))
print("Atributo size: {}".format(accept.get_attribute('size')))

accept.click()

Etiqueta: button
Texto de la etiqueta: ACEPTO
Atributo mode: primary
Atributo size: large


Hacemos la búsqueda de una película

In [270]:
search = driver.find_element(By.TAG_NAME, 'input')
search

<selenium.webdriver.remote.webelement.WebElement (session="ed48f57b61d6047db7a7188e283c0a18", element="C85EE262BD2FD55CDE54694211635F43_element_27")>

In [271]:
search = driver.find_element(By.ID, 'top-search-input-2')
search

<selenium.webdriver.remote.webelement.WebElement (session="ed48f57b61d6047db7a7188e283c0a18", element="C85EE262BD2FD55CDE54694211635F43_element_27")>

In [272]:
search.send_keys("Harry Potter")

In [273]:
search.clear()

In [274]:
def imprimir_poco_a_poco(texto, velocidad=0.15):
    for caracter in texto:
        search.send_keys(caracter)  
        time.sleep(velocidad)  # Pausa entre caracteres

In [275]:
#Introducimos el nombre de la película a buscar
imprimir_poco_a_poco('Oblivion')


search.send_keys(Keys.ENTER)

Entramos en el primer resultado

In [276]:
movie = driver.find_elements(By.CLASS_NAME, "se-it")[0]
movie = driver.find_element(By.CLASS_NAME, "se-it")

#'div.mc-title a'
#'div.mc-info-container a'

#Accedemos mediante un selector CSS al elemento a, que contiene el enlace (texto)
url = movie.find_element(By.CSS_SELECTOR, 'div.mc-title a')

#Accedemos mediante un selector CSS al elemento a, que contiene el enlace (texto) desde el padre
url = movie.find_element(By.CSS_SELECTOR, 'div.mc-info-container a')

In [277]:
#Accedemos mediante la clase al elemento a, que contiene el enlace (texto)
url_titulo = movie.find_element(By.CLASS_NAME, 'mc-title')
#url=url_titulo.find_element(By.TAG_NAME, 'a')
url_titulo.text


'Oblivion'

In [278]:
url = movie.find_element(By.CSS_SELECTOR, 'div.mc-poster a')



In [279]:
movie.screenshot('pelicula.png')

True

In [280]:
link = url.get_attribute('href')
print(link)

https://www.filmaffinity.com/es/film618375.html


In [281]:
url.click()

In [282]:
#Volver atras
driver.back()
time.sleep(1)
#Volver adelante
driver.forward()

Vamos a sacar los datos principales de la película:

In [283]:
data = driver.find_element(By.CLASS_NAME, 'movie-info')
dts = data.find_elements(By.TAG_NAME, 'dt')
dds = data.find_elements(By.TAG_NAME, 'dd')

#len(dts) == len(dds)
movie_dict = dict()
for i in range(len(dts)):
    movie_dict[dts[i].text] = dds[i].text
    print(f"{dts[i].text} - {dds[i].text}")

Título original - Oblivion
Año - 2013
Duración - 126 min.
País -  Estados Unidos
Dirección - Joseph Kosinski
Guion - Joseph Kosinski, Michael Arndt, Karl Gajdusek. Cómic: Joseph Kosinski, Arvid Nelson
Reparto - Tom Cruise
Andrea Riseborough
Olga Kurylenko
Morgan Freeman
Nikolaj Coster-Waldau
Zoe Bell
Música - Anthony Gonzalez, M83, Joseph Trapanese
Fotografía - Claudio Miranda
Compañías - Universal Pictures, Chernin Entertainment, Relativity Studios, Monolith Pictures, Radical Studios
Género - Ciencia ficción. Intriga | Futuro postapocalíptico. Distopía. Cómic
Sinopsis - Año 2073. Hace más de 60 años la Tierra fue atacada; se ganó la guerra, pero la mitad del planeta quedó destruido, y todos los seres humanos fueron evacuados. Jack Harper (Tom Cruise), un antiguo marine, es uno de los últimos hombres que la habitan. Es un ingeniero de Drones que participa en una operación para extraer los recursos vitales del planeta. Su misión consiste en patrullar diariamente los cielos. Un día, resc

In [284]:
movie_dict

{'Título original': 'Oblivion',
 'Año': '2013',
 'Duración': '126 min.',
 'País': ' Estados Unidos',
 'Dirección': 'Joseph Kosinski',
 'Guion': 'Joseph Kosinski, Michael Arndt, Karl Gajdusek. Cómic: Joseph Kosinski, Arvid Nelson',
 'Reparto': 'Tom Cruise\nAndrea Riseborough\nOlga Kurylenko\nMorgan Freeman\nNikolaj Coster-Waldau\nZoe Bell',
 'Música': 'Anthony Gonzalez, M83, Joseph Trapanese',
 'Fotografía': 'Claudio Miranda',
 'Compañías': 'Universal Pictures, Chernin Entertainment, Relativity Studios, Monolith Pictures, Radical Studios',
 'Género': 'Ciencia ficción. Intriga | Futuro postapocalíptico. Distopía. Cómic',
 'Sinopsis': 'Año 2073. Hace más de 60 años la Tierra fue atacada; se ganó la guerra, pero la mitad del planeta quedó destruido, y todos los seres humanos fueron evacuados. Jack Harper (Tom Cruise), un antiguo marine, es uno de los últimos hombres que la habitan. Es un ingeniero de Drones que participa en una operación para extraer los recursos vitales del planeta. Su mi

#### Creamos un dataframe

In [285]:
df= pd.DataFrame([movie_dict]) #es decir, mediante una lista de diccionarios (1 diccionario)


In [286]:
df

,Título original,Año,Duración,País,Dirección,Guion,Reparto,Música,Fotografía,Compañías,Género,Sinopsis
0,Oblivion,2013,126 min.,Estados Unidos,Joseph Kosinski,"Joseph Kosinski, Michael Arndt, Karl Gajdusek....",Tom Cruise\nAndrea Riseborough\nOlga Kurylenko...,"Anthony Gonzalez, M83, Joseph Trapanese",Claudio Miranda,"Universal Pictures, Chernin Entertainment, Rel...",Ciencia ficción. Intriga | Futuro postapocalíp...,Año 2073. Hace más de 60 años la Tierra fue at...


In [289]:
driver.back()
search = driver.find_element(By.ID, 'top-search-input-2')
search.clear()
imprimir_poco_a_poco('Rebel Moon')


search.send_keys(Keys.ENTER)
movie = driver.find_elements(By.CLASS_NAME, "se-it")[0]
movie = driver.find_element(By.CLASS_NAME, "se-it")

#'div.mc-title a'
#'div.mc-info-container a'

#Accedemos mediante un selector CSS al elemento a, que contiene el enlace (texto)
url = movie.find_element(By.CSS_SELECTOR, 'div.mc-title a')
url.click()
data = driver.find_element(By.CLASS_NAME, 'movie-info')
dts = data.find_elements(By.TAG_NAME, 'dt')
dds = data.find_elements(By.TAG_NAME, 'dd')

#len(dts) == len(dds)
movie_dict = dict()
for i in range(len(dts)):
    movie_dict[dts[i].text] = dds[i].text
    print(f"{dts[i].text} - {dds[i].text}")
df = pd.concat([df, pd.DataFrame([movie_dict])])


Título original - Rebel Moon: A Child of Fire
Año - 2023
Duración - 125 min.
País -  Estados Unidos
Dirección - Zack Snyder
Guion - Zack Snyder, Kurt Johnstad, Shay Hatten. Historia: Zack Snyder
Reparto - Sofia Boutella
Djimon Hounsou
Ed Skrein
Michiel Huisman
Bae Doona
Ray Fisher
Música - Junkie XL
Fotografía - Zack Snyder
Compañías - Grand Electric, The Stone Quarry. Distribuidora: Netflix
Género - Ciencia ficción. Fantástico. Acción. Aventuras | Aventura espacial. Cine épico
Sinopsis - Cuando una pacífica colonia en los confines de la galaxia se ve amenazada por un poder tiránico, su mejor baza para sobrevivir es la misteriosa Kora. Esta reúne a un grupo dispuesto a luchar, formado por forasteros, insurgentes, campesinos y huérfanos de guerra de varios planetas unidos por una causa común: la redención y la venganza. Mientras la sombra de todo un reino se cierne sobre la más improbable de las lunas, se libra una batalla por el destino de una galaxia y se forja un nuevo ejército de hé

,Título original,Año,Duración,País,Dirección,Guion,Reparto,Música,Fotografía,Compañías,Género,Sinopsis,Grupos
0,Oblivion,2013,126 min.,Estados Unidos,Joseph Kosinski,"Joseph Kosinski, Michael Arndt, Karl Gajdusek....",Tom Cruise\nAndrea Riseborough\nOlga Kurylenko...,"Anthony Gonzalez, M83, Joseph Trapanese",Claudio Miranda,"Universal Pictures, Chernin Entertainment, Rel...",Ciencia ficción. Intriga | Futuro postapocalíp...,Año 2073. Hace más de 60 años la Tierra fue at...,NaN
0,Dune,2021,155 min.,Estados Unidos,Denis Villeneuve,"Eric Roth, Denis Villeneuve, Jon Spaihts. Nove...",Timothée Chalamet\nRebecca Ferguson\nOscar Isa...,Hans Zimmer,Greig Fraser,"Legendary Pictures, Villeneuve Films, Warner B...",Ciencia ficción. Aventuras. Drama | Aventura e...,"En el Año 10191 el desértico planeta Arrakis, ...",Dune
0,Rebel Moon: A Child of Fire,2023,125 min.,Estados Unidos,Zack Snyder,"Zack Snyder, Kurt Johnstad, Shay Hatten. Histo...",Sofia Boutella\nDjimon Hounsou\nEd Skrein\nMic...,Junkie XL,Zack Snyder,"Grand Electric, The Stone Quarry. Distribuidor...",Ciencia ficción. Fantástico. Acción. Aventuras...,Cuando una pacífica colonia en los confines de...,NaN


In [291]:

df.reset_index(inplace=True)
df

,index,Título original,Año,Duración,País,Dirección,Guion,Reparto,Música,Fotografía,Compañías,Género,Sinopsis,Grupos
0,0,Oblivion,2013,126 min.,Estados Unidos,Joseph Kosinski,"Joseph Kosinski, Michael Arndt, Karl Gajdusek....",Tom Cruise\nAndrea Riseborough\nOlga Kurylenko...,"Anthony Gonzalez, M83, Joseph Trapanese",Claudio Miranda,"Universal Pictures, Chernin Entertainment, Rel...",Ciencia ficción. Intriga | Futuro postapocalíp...,Año 2073. Hace más de 60 años la Tierra fue at...,NaN
1,0,Dune,2021,155 min.,Estados Unidos,Denis Villeneuve,"Eric Roth, Denis Villeneuve, Jon Spaihts. Nove...",Timothée Chalamet\nRebecca Ferguson\nOscar Isa...,Hans Zimmer,Greig Fraser,"Legendary Pictures, Villeneuve Films, Warner B...",Ciencia ficción. Aventuras. Drama | Aventura e...,"En el Año 10191 el desértico planeta Arrakis, ...",Dune
2,0,Rebel Moon: A Child of Fire,2023,125 min.,Estados Unidos,Zack Snyder,"Zack Snyder, Kurt Johnstad, Shay Hatten. Histo...",Sofia Boutella\nDjimon Hounsou\nEd Skrein\nMic...,Junkie XL,Zack Snyder,"Grand Electric, The Stone Quarry. Distribuidor...",Ciencia ficción. Fantástico. Acción. Aventuras...,Cuando una pacífica colonia en los confines de...,NaN


#### EXTRA

Abrir nueva ventana:

In [258]:
driver.execute_script('window.open("");')

Movernos a otra ventana

In [259]:
driver.switch_to.window(driver.window_handles[-1])

Cerrar ventana

In [261]:
driver.close()

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=120.0.6099.199)
Stacktrace:
	GetHandleVerifier [0x01306EE3+174339]
	(No symbol) [0x01230A51]
	(No symbol) [0x00F46FF6]
	(No symbol) [0x00F2EFE7]
	(No symbol) [0x00F9B53B]
	(No symbol) [0x00F9FC7B]
	(No symbol) [0x00F96FD3]
	(No symbol) [0x00F71034]
	(No symbol) [0x00F71F8D]
	GetHandleVerifier [0x013A4B1C+820540]
	sqlite3_dbdata_init [0x014653EE+653550]
	sqlite3_dbdata_init [0x01464E09+652041]
	sqlite3_dbdata_init [0x014597CC+605388]
	sqlite3_dbdata_init [0x01465D9B+656027]
	(No symbol) [0x0123FE6C]
	(No symbol) [0x012383B8]
	(No symbol) [0x012384DD]
	(No symbol) [0x01225818]
	BaseThreadInitThunk [0x75C07BA9+25]
	RtlInitializeExceptionChain [0x778DBD2B+107]
	RtlClearBits [0x778DBCAF+191]
